# ReadMe

Here is a list of some of the changes I made to get something to work. This does not mean I did all the work for you, I merely adjusted some things around to run better and fixed some logical bugs. 

1. I separated everything into its own cell. You can run each cell individually, and if you want to test a new part of the code you will not have to re-run all the previous code. 
2. Do not try to call and separate your data in every function. It is good practice to have a separate file for preparing the data and exporting to a new CSV file. All you would have to do from there is load your pre-made CSV into the model file. Lot cleaner. The data prep does not need to be run more than once. 
3. When putting data into a model, the data needs to be the same shape. What I mean by that is you cannot put a row of 50 items and a row of 2000 items into the same data frame. You tried something simular when defining x_warp, x_war, etc.  There are methods of making the data the same size, you can either trim the data down or you can pad the data up with a common variable (usually NaN or 0). 
4. To test your models, I had to choose random columns from one of your cvs files to use. Each file was a different size, therefore I could not use multiple files within the same model.
5. You had all of your models working expect the KNN. It mainly comes down to the data that I was using. With a proper dataframe passed into it, it may work.  
6. Because of the data being randomly chosen, I was not able to build a scatter plot off of it. Once you get the dataframes fixed and run through the model, then you can start looking into getting some visualization. The data that I put through the model will not make any sense to the human eye and may end up causing more problems and not working at all. 

# Importing Libraries

In [22]:
import pandas as pd
from DataParser import *
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_absolute_error,r2_score, mean_squared_error
import plotly.express as px
import re

# Data Preperation

## Data Preperation Function

In [97]:
def data_preparation(hitter_data, hitter_pred_data, pitcher_data, pitcher_pred_data, war_values):
# 	hitter_data = clean_sorted_hitter()
# 	hitter_pred_data = clean_warp_hitter()
# 	pitcher_data = clean_sorted_pitcher()
# 	pitcher_pred_data = clean_warp_pitcher()
# 	#defensive_values = clean_defensive_players() 
# 	#baserunning_values = clean_sorted_baserunning() 
# 	war_values = clean_war()

    x_warp = pd.DataFrame()
    y_warp = pd.DataFrame()
    x_war = pd.DataFrame()
    y_war = pd.DataFrame()
    #multiple linear regression where x is composed of multiple variables

    lastName = r'[\S]*'
    for index,row in hitter_pred_data.iterrows():
        name = row['Name'].split()[1]
        print('name: ', name)
        for i in hitter_data['Hitters']:
            print(i)
        foundName = re.findall(lastName, hitter_data['Hitters'][index])
        if len(foundName) <= 2:
            continue
        print('Hitters: ', foundName[2])
        #print(hitter_data['Hitters'].values[index])
        if foundName==name:
            print('inside if statement')
            x_warp.append(hitter_data[name,['K','BB','AVG','OBP','SLG']])
            x_war.append(hitter_data[name,['K','BB','AVG','OBP','SLG']])
            # y is pulled from a separate database that I pulled to actually get the x variables to predict a "performance number" rather than a correlation between two statistics 
            y_warp.append(hitter_pred_data['WARP'])
            y_war.append(war_values['Total War'])
        #if defensive_values.get_val(name) != "No record found":
        #	x_war+= defensive_values.get_val(name)
        #if baserunning_values.get_val(name) != "No record found":
    #		x_war+= baserunning_values.get_val(name)
    print("x_Warp",x_warp)
    print("y_warp",y_warp)

    # add additional factors based off of rows in the relevant cavs, (player.csv for players, pitcher.csv for pitchers)
    x_warp_train, x_warp_test, y_warp_train, y_warp_test = train_test_split(x_warp, y_warp, test_size=.25,random_state=1)
    x_war_train, x_war_test, y_war_train, y_war_test = train_test_split(x_war, y_war, test_size = .25,train_size = .75, random_state=1)
# 	a_warp = []
# 	b_warp = []
# 	a_war=[]
# 	b_war = []
    for row in pitcher_pred_data.iterrows():
        name = row['Name']
        if pitcher_data['Pitchers'].values[0]==name:
            a_warp = pitcher_data[name,['IP', 'BB','K','HR','ERA']]
            # y is pulled from a separate database that I pulled to actually get the x variables to predict a "performance number" rather than a correlation between two statistics 
            b_warp = pitcher_pred_data['WARP']
            b_war = war_values['Primary WAR']
    a_warp_train, a_warp_test, b_warp_train, b_warp_test = train_test_split(a_warp,b_warp, test_size=.25,train_size = .75,random_state=1)
    a_war_train, a_war_test, b_war_train, b_war_test = train_test_split(a_war,b_war, test_size=.25,train_size = .75,random_state=1)


    return (x_warp_train,x_warp_test,y_warp_train,y_warp_test,x_war_train,x_war_test,y_war_train,y_war_test,a_warp_train,a_warp_test,b_warp_train,b_warp_test,a_war_train,a_war_test, b_war_train, b_war_test)

## Loading in all CSV files

In [18]:
# Loading in all the data
hitter_data = clean_sorted_hitter()
hitter_pred_data = clean_warp_hitter()
pitcher_data = clean_sorted_pitcher()
pitcher_pred_data = clean_warp_pitcher()
#defensive_values = clean_defensive_players() 
#baserunning_values = clean_sorted_baserunning() 
war_values = clean_war()

C:\Users\tuffd\AppData\Local\Temp/ipykernel_18032/4042163456.py:1: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  hitter_data = clean_sorted_hitter()
C:\Users\tuffd\AppData\Local\Temp/ipykernel_18032/4042163456.py:3: DtypeWarning: Columns (9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  pitcher_data = clean_sorted_pitcher()


## Splitting data into Train Test Split

In [157]:
# Splitting all the data
x_warp = hitter_pred_data[['G', 'PA', 'AB', 'R', 'HR', 'RBI', 'SB']].values
y_warp = hitter_pred_data['WARP'].values
x_warp_train, x_warp_test, y_warp_train, y_warp_test = train_test_split(x_warp, y_warp, test_size=.25,random_state=1)
x_war = hitter_data[['AB', 'R', 'H', 'RBI', 'BB', 'K', '#P']].values
y_war = hitter_data['AVG'].values
x_war_train, x_war_test, y_war_train, y_war_test = train_test_split(x_war, y_war, test_size = .25,train_size = .75, random_state=1)


a_warp = hitter_pred_data[['G', 'PA', 'AB', 'R', 'HR', 'RBI', 'SB']].values
a_war = hitter_data[['AB', 'R', 'H', 'RBI', 'BB', 'K', '#P']].values
b_warp = hitter_pred_data['WARP'].values
b_war = hitter_data['AVG'].values

a_warp_train, a_warp_test, b_warp_train, b_warp_test = train_test_split(a_warp,b_warp, test_size=.25,train_size = .75,random_state=1)
a_war_train, a_war_test, b_war_train, b_war_test = train_test_split(a_war,b_war, test_size=.25,train_size = .75,random_state=1)

# Build DataFrame - Break into Train Test Split

# Linear Model

## Function Definition 

In [158]:
def performancelinear(x_warp_train,x_warp_test,y_warp_train,y_warp_test,x_war_train,x_war_test,y_war_train,y_war_test,a_warp_train,a_warp_test,b_warp_train,b_warp_test,a_war_train, a_war_test, b_war_train, b_war_test):
# 	x_warp_train,x_warp_test,y_warp_train,y_warp_test,x_war_train,x_war_test,y_war_train,y_war_test,a_warp_train,a_warp_test,b_warp_train,b_warp_test,a_war_train, a_war_test, b_war_train, b_war_test = data_preparation()
    regressor = LinearRegression()
    regressor.fit(x_warp_train, y_warp_train)
    y_warp_pred = regressor.predict(x_warp_test)
    print("R2Linear - hitter WARP correlation: " + str(r2_score(y_warp_test,y_warp_pred)) + ", RMSELinear - hitter WARP correlation: " + str(mean_squared_error(y_warp_test,y_warp_pred)))
#     print('x_war_train: ', x_war_train)
#     print('y_war_train: ', y_war_train)
#     regressor.fit(x_war_train, y_war_train)
#     y_war_pred = regressor.predict(x_war_test)
#     print("R2Linear - hitter WAR correlation: " + r2_score(y_war_test,y_war_pred) + ", RMSELinear - hitter WAR correlation: " + mean_squared_error(y_war_test,y_war_pred))
#     regressor.fit(a_warp_train, b_warp_train)
#     b_warp_pred = regressor.predict(a_warp_test)
#     print("R2Linear - pitcher WARP correlation: " + r2_score(b_warp_test,b_warp_pred) + ", RMSELinear - pitcher WARP correlation: " + mean_squared_error(b_warp_test,b_warp_pred))
#     regressor.fit(a_war_train, b_war_train)
#     b_war_pred = regressor.predict(a_war_test)
#     print("R2Linear - pitcher WAR correlation: " + r2_score(b_war_test,b_war_pred) + ", RMSELinear - pitcher WAR correlation: " + mean_squared_error(b_war_test,b_war_pred))
#     figure  = px.scatter(y_warp_pred,y_warp_test, x = 'G', y = 'Actual Performance Hitter(WARP)', hover_name = y_warp_test-y_warp_pred, title = 'Actual Performance Hitter vs. Pred. Performance')
#     figure.show()
#     figure  = px.scatter(y_war_pred,y_war_test, x = 'Predicted Performance Hitter(Linear)', y = 'Actual Performance Hitter(WAR)', hover_name = y_war_test-y_war_pred, title = 'Actual Performance Hitter vs. Pred. Performance')
#     figure.show()
#     figure  = px.scatter(b_warp_pred,b_warp_test, x = 'Predicted Performance Pitcher(Linear)', y = 'Actual Performance Pitcher(WARP)', hover_name = b_warp_test-b_warp_pred, title = 'Actual Performance Pitcher vs. Pred. Performance')
#     figure.show()
#     figure  = px.scatter(b_war_pred,b_war_test, x = 'Predicted Performance Pitcher(Linear)', y = 'Actual Performance Pitcher(WAR)', hover_name = b_war_test-b_war_pred, title = 'Actual Performance Pitcher vs. Pred. Performance')
#     figure.show()    

## Calling Function

In [159]:
performancelinear(x_warp_train,x_warp_test,y_warp_train,y_warp_test,x_war_train,x_war_test,y_war_train,y_war_test,a_warp_train,a_warp_test,b_warp_train,b_warp_test,a_war_train, a_war_test, b_war_train, b_war_test)


R2Linear - hitter WARP correlation: 0.801574018632002, RMSELinear - hitter WARP correlation: 0.3776173538390747


# Lasso Model

## Function Definition

In [160]:
def performancelasso(x_warp_train,x_warp_test,y_warp_train,y_warp_test,x_war_train,x_war_test,y_war_train,y_war_test,a_warp_train,a_warp_test,b_warp_train,b_warp_test,a_war_train, a_war_test, b_war_train, b_war_test):
# 	x_warp_train,x_warp_test,y_warp_train,y_warp_test,x_war_train,x_war_test,y_war_train,y_war_test,a_warp_train,a_warp_test,b_warp_train,b_warp_test,a_war_train, a_war_test, b_war_train, b_war_test = data_preparation()
    regressor = Lasso()
    regressor.fit(x_warp_train, y_warp_train)
    y_warp_pred = regressor.predict(x_warp_test)
    print("R2Lasso - hitter WARP correlation: " + str(r2_score(y_warp_test,y_warp_pred)) + ", RMSELasso - hitter WARP correlation: " + str(mean_squared_error(y_warp_test,y_warp_pred)))
#     regressor.fit(x_war_train, y_war_train)
#     y_war_pred = regressor.predict(x_war_test)
#     print("R2Lasso - hitter WAR correlation: " + r2_score(y_war_test,y_war_pred) + ", RMSELasso - hitter WAR correlation: " + mean_squared_error(y_war_test,y_war_pred))
#     regressor.fit(a_warp_train, b_warp_train)
#     b_warp_pred = regressor.predict(a_warp_test)
#     print("R2Lasso - pitcher WARP correlation: " + r2_score(b_warp_test,b_warp_pred) + ", RMSELasso - pitcher WARP correlation: " + mean_squared_error(b_warp_test,b_warp_pred))
#     regressor.fit(a_war_train, b_war_train)
#     b_war_pred = regressor.predict(a_war_test)
#     print("R2Lasso - pitcher WAR correlation: " + r2_score(b_war_test,b_war_pred) + ", RMSELasso - pitcher WAR correlation: " + mean_squared_error(b_war_test,b_war_pred))
#     figure  = px.scatter(y_warp_pred,y_warp_test, x = 'Predicted Performance Hitter(Lasso)', y = 'Actual Performance Hitter(WARP)', hover_name = y_warp_test-y_warp_pred, title = 'Actual Performance Hitter vs. Pred. Performance')
#     figure.show()
#     figure  = px.scatter(y_war_pred,y_war_test, x = 'Predicted Performance Hitter(Lasso)', y = 'Actual Performance Hitter(WAR)', hover_name = y_war_test-y_war_pred, title = 'Actual Performance Hitter vs. Pred. Performance')
#     figure.show()
#     figure  = px.scatter(b_warp_pred,b_warp_test, x = 'Predicted Performance Pitcher(Lasso)', y = 'Actual Performance Pitcher(WARP)', hover_name = b_warp_test-b_warp_pred, title = 'Actual Performance Pitcher vs. Pred. Performance')
#     figure.show()
#     figure  = px.scatter(b_war_pred,b_war_test, x = 'Predicted Performance Pitcher(Lasso)', y = 'Actual Performance Pitcher(WAR)', hover_name = b_war_test-b_war_pred, title = 'Actual Performance Pitcher vs. Pred. Performance')
#     figure.show()

## Calling Function

In [161]:
performancelasso(x_warp_train,x_warp_test,y_warp_train,y_warp_test,x_war_train,x_war_test,y_war_train,y_war_test,a_warp_train,a_warp_test,b_warp_train,b_warp_test,a_war_train, a_war_test, b_war_train, b_war_test)


R2Lasso - hitter WARP correlation: 0.7913286374097127, RMSELasso - hitter WARP correlation: 0.3971149706307906


# Elastic Model

## Function Definition

In [162]:
def performanceelasticnet(x_warp_train,x_warp_test,y_warp_train,y_warp_test,x_war_train,x_war_test,y_war_train,y_war_test,a_warp_train,a_warp_test,b_warp_train,b_warp_test,a_war_train, a_war_test, b_war_train, b_war_test):
# 	x_warp_train,x_warp_test,y_warp_train,y_warp_test,x_war_train,x_war_test,y_war_train,y_war_test,a_warp_train,a_warp_test,b_warp_train,b_warp_test,a_war_train, a_war_test, b_war_train, b_war_test = data_preparation()
    regressor = ElasticNet()
    regressor.fit(x_warp_train,y_warp_train)
    y_warp_pred = regressor.predict(x_warp_test)
    print("R2Elastic - hitter WARP correlation: " + str(r2_score(y_warp_test,y_warp_pred)) + ", RMSEElastic - hitter WARP correlation: " + str(mean_squared_error(y_warp_test,y_warp_pred)))
#     regressor.fit(x_war_train,y_war_train)
#     y_war_pred = regressor.predict(x_war_test)
#     print("R2Elastic - hitter WAR correlation: " + r2_score(y_war_test,y_war_pred) + ", RMSEElastic - hitter WAR correlation: " + mean_squared_error(y_war_test,y_war_pred))
#     regressor.fit(a_warp_train, b_warp_train)
#     b_warp_pred = regressor.predict(a_warp_test)
#     print("R2Elastic - pitcher WARP correlation: " + r2_score(b_warp_test,b_warp_pred) + ", RMSEElastic - pitcher WARP correlation: " + mean_squared_error(b_warp_test,b_warp_pred))
#     regressor.fit(a_war_train, b_war_train)
#     b_war_pred = regressor.predict(a_war_test)
#     print("R2Elastic - pitcher WAR correlation: " + r2_score(b_war_test,b_war_pred) + ", RMSEElastic - pitcher WAR correlation: " + mean_squared_error(b_war_test,b_war_pred))
#     figure  = px.scatter(y_warp_pred,y_warp_test, x = 'Predicted Performance Hitter(Elastic Net)', y = 'Actual Performance Hitter(WARP)', hover_name = y_warp_test-y_warp_pred, title = 'Actual Performance Hitter vs. Pred. Performance')
#     figure.show()
#     figure  = px.scatter(y_war_pred,y_war_test, x = 'Predicted Performance Hitter(Elastic Net)', y = 'Actual Performance Hitter(WAR)', hover_name = y_war_test-y_war_pred, title = 'Actual Performance Hitter vs. Pred. Performance')
#     figure.show()
#     figure  = px.scatter(b_warp_pred,b_warp_test, x = 'Predicted Performance Pitcher(Elastic Net)', y = 'Actual Performance Pitcher(WARP)', hover_name = y_warp_test-y_warp_pred, title = 'Actual Performance Pitcher vs. Pred. Performance')
#     figure.show()
#     figure  = px.scatter(b_war_pred,b_war_test, x = 'Predicted Performance Pitcher(Elastic Net)', y = 'Actual Performance Pitcher(WAR)', hover_name = y_war_test-y_war_pred, title = 'Actual Performance Pitcher vs. Pred. Performance')
#     figure.show()

## Calling Function

In [163]:
performanceelasticnet(x_warp_train,x_warp_test,y_warp_train,y_warp_test,x_war_train,x_war_test,y_war_train,y_war_test,a_warp_train,a_warp_test,b_warp_train,b_warp_test,a_war_train, a_war_test, b_war_train, b_war_test)


R2Elastic - hitter WARP correlation: 0.8003410758550977, RMSEElastic - hitter WARP correlation: 0.3799637229266293


C:\Users\tuffd\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.07418564858173227, tolerance: 0.06473891066282422
  model = cd_fast.enet_coordinate_descent(


# KNN Model

## Function Definition

In [164]:
def performanceknnvisualization(x_warp_train,x_warp_test,y_warp_train,y_warp_test,x_war_train,x_war_test,y_war_train,y_war_test,a_warp_train,a_warp_test,b_warp_train,b_warp_test,a_war_train, a_war_test, b_war_train, b_war_test):
    model_warp_hitter = KNeighborsClassifier(n_neighbors = 3)
    model_warp_hitter.fit(x_warp_train,y_warp_train)
#     model_warp_pitcher = KNeighborsClassifier(n_neighbors = 3)
#     model_warp_pitcher.fit(a_warp_train,b_warp_train)
#     model_war_hitter = KNeighborsClassifier(n_neighbors = 3)
#     model_war_hitter.fit(x_war_train,y_war_train)
#     model_war_pitcher = KNeighborsClassifier(n_neighbors = 3)
#     model_war_pitcher.fit(a_war_train,b_war_train)
    print("Accuracy Score - KNN WARP hitters: " + model_warp_hitter.score(x_warp_test,y_warp_test))
#     print("Accuracy Score - KNN WARP pitchers: " + model_warp_pitcher.score(a_warp_test,b_warp_test))
#     print("Accuracy Score - KNN WAR hitters: " + model_war_hitter.score(x_war_test,y_war_test))
#     print("Accuracy Score - KNN WAR pitchers: " + model_war_pitcher.score(a_war_test,b_war_test))

## Calling Function

In [165]:
performanceknnvisualization(x_warp_train,x_warp_test,y_warp_train,y_warp_test,x_war_train,x_war_test,y_war_train,y_war_test,a_warp_train,a_warp_test,b_warp_train,b_warp_test,a_war_train, a_war_test, b_war_train, b_war_test)


ValueError: Unknown label type: 'continuous'